# Generate Random numbers nPr with r between 2 values

In [1]:
#generate random permutations nPr = P(n,r) = n!/(n-r)!
import random
import itertools
import pandas as pd
import numpy as np

#constituency = 'Meath2002'
#constituency = 'DublinNorth2002'
constituency = 'DublinWest2002'
n = 9 #candidates (Meath 14, Dublin North 12, Dublin West 9)
r = 7  #preference cast
o = 10000 #number of permutations to be generated (25000, 20000, 10000)
nlist = list(range(1, n+1)) 

randGenPer = random.sample(list(itertools.permutations(nlist, r)), o)

## Convert the permutation lists into a list of votes

In [2]:
#function to convert a permutation list into vote
def listToVote(listgen, num):
    voteList=[]
    for j in range(num):
        voteList.append(np.NaN)
    #print(voteList)

    for k in range(len(listgen)):
        voteList[listgen[k]-1] =k+1 
    return(voteList)
    

In [3]:
#convert each of the permutations generated into a vote
votesGen=[]
for row in randGenPer:
    #print(row)
    votesGen.append(listToVote(row,n))
#print(votesGen)

## Make data frame from list and write to file

In [4]:
#generate data frame from list
dfRandGenPer = pd.DataFrame(votesGen)
#drop all dublicate rows keeping last one
dfRandGenPer.drop_duplicates(keep='last', inplace=True)

In [5]:
# float_format='%.0f' to get rid of the decimal 1 not 1.0
#write to csv file
dfRandGenPer.to_csv('../data/'+constituency+'-'+str(n)+'P'+str(r)+'-r-'+str(o)+'Generated.csv', 
                    float_format='%.0f')
